In [1]:
%pip install flask
%pip install structlog



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import logging
import uuid
import time
from flask import Flask, g, request

app = Flask(__name__)

# 基本ロガーの設定
formatter = logging.Formatter(
    "%(asctime)s - %(levelname)s - [user: %(user)s] [request_id: %(request_id)s] - %(message)s"
)
handler = logging.StreamHandler()
handler.setFormatter(formatter)

base_logger = logging.getLogger("server_logger")
base_logger.setLevel(logging.INFO)
base_logger.addHandler(handler)


# LoggerAdapter の定義
class ContextLogger(logging.LoggerAdapter):
    def process(self, msg, kwargs):
        # `g` からユーザー情報を取得し、extra に追加
        extra = kwargs.get("extra", {})
        extra["user"] = getattr(g, "user", "unknown")
        extra["request_id"] = getattr(g, "trace_id", "N/A")
        kwargs["extra"] = extra
        return msg, kwargs


logger = ContextLogger(base_logger, {})


@app.before_request
def before_request():
    # リクエストごとのコンテキスト情報を設定
    g.user = request.headers.get("X-User", "anonymous")
    g.trace_id = uuid.uuid4().hex


@app.route("/")
def index():
    logger.info("Starting processing request")
    time.sleep(1)  # 1秒待機
    logger.info("Finished processing request")
    return "Hello, Flask!"


if __name__ == "__main__":
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
2024-11-17 09:19:07,185 - INFO - [user: anonymous] [request_id: default-id] - Processing request
127.0.0.1 - - [17/Nov/2024 09:19:07] "GET / HTTP/1.1" 200 -
2024-11-17 09:19:07,300 - INFO - [user: anonymous] [request_id: default-id] - Processing request
127.0.0.1 - - [17/Nov/2024 09:19:07] "GET / HTTP/1.1" 200 -
2024-11-17 09:19:07,410 - INFO - [user: anonymous] [request_id: default-id] - Processing request
127.0.0.1 - - [17/Nov/2024 09:19:07] "GET / HTTP/1.1" 200 -


In [11]:
import logging
import uuid
import time
from flask import Flask, request
import structlog

app = Flask(__name__)

# 標準のロギング設定
logging.basicConfig(format="%(message)s", level=logging.INFO)

# structlog の設定
structlog.configure(
    processors=[
        structlog.contextvars.merge_contextvars,  # contextvars の値をログにマージ
        structlog.processors.CallsiteParameterAdder(
            parameters=[
                structlog.processors.CallsiteParameter.FILENAME,  # Adds the filename
                structlog.processors.CallsiteParameter.LINENO,  # Adds the line number
            ]
        ),
        structlog.processors.format_exc_info,
        structlog.processors.TimeStamper(fmt="iso"),
        structlog.processors.JSONRenderer(),
    ],
    logger_factory=structlog.stdlib.LoggerFactory(),
    wrapper_class=structlog.make_filtering_bound_logger(logging.INFO),
)

logger = structlog.get_logger("server_logger")


@app.before_request
def before_request():
    user = request.headers.get("X-User", "anonymous")
    trace_id = uuid.uuid4().hex
    # contextvars にコンテキスト情報をバインド
    structlog.contextvars.bind_contextvars(user=user, trace_id=trace_id)


@app.route("/")
def index():
    logger.info("Starting processing request")
    time.sleep(1)  # 1秒待機
    logger.info("Finished processing request")

    try:
        # 例外を発生させるコード
        1 / 0
    except Exception:
        # 例外情報を含めてログを出力
        logger.exception("例外が発生しました")

    return "Hello, Flask!"


if __name__ == "__main__":
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
{"event": "Starting processing request", "user": "anonymous", "trace_id": "1f6c52dc3fee43de9da314cf84a35d25", "filename": "4091127567.py", "lineno": 43, "timestamp": "2024-11-17T02:31:46.183251Z"}
{"event": "Finished processing request", "user": "anonymous", "trace_id": "1f6c52dc3fee43de9da314cf84a35d25", "filename": "4091127567.py", "lineno": 45, "timestamp": "2024-11-17T02:31:47.190477Z"}
{"event": "\u4f8b\u5916\u304c\u767a\u751f\u3057\u307e\u3057\u305f", "user": "anonymous", "trace_id": "1f6c52dc3fee43de9da314cf84a35d25", "filename": "4091127567.py", "lineno": 52, "exception": "Traceback (most recent call last):\n  File \"/var/folders/27/6hk2mfzs5vv9ysvns0ydhx2m0000gn/T/ipykernel_36596/4091127567.py\", line 49, in index\n    1 / 0\nZeroDivisionError: division by zero", "timestamp": "2024-11-17T02:31:47.192677Z"}
127.0.0.1 - - [17/Nov/2024 11:31:47] "GET / HTTP/1.1" 200 -
